# YOLOv5 XAI Analysis Using EigenCAM

## Input variables

In [2]:
import os

# Input Image Directory
input_dir = r'\\wsl.localhost\Debian\home\akashanil\Projects\yolov5_hyouka\test_datasets\new\v360_test_dataset\images'

# Target Layers (results are averaged across layers when multiple specified)
# YOLOv5 Target layers: 4, 6, 9, 17, 20, 23
target_layers = [-2]

# Model Weights
weights_path = r'\\wsl.localhost\Debian\home\akashanil\Projects\yolov5_hyouka\models\hyouka\241019072903_runs_train_2024_10_tc5_dataset_11\content\yolov5\runs\train\exp\weights\best.pt'

# Save Path
save_path = r'D:\Akash\Work\AI\XAI\XAI_save'

# Save Directory Name
# save_dir_name = 'results'
save_dir_name = os.path.basename(input_dir) + '_EigenCAM'

# Save Image Prefix
img_prefix = ''
# Save Image Suffix
img_suffix = '_EigenCAM'

# Install requirements

In [2]:
%pip install -U grad-cam numpy Pillow torch>=1.7.1 torchvision>=0.8.2 ttach tqdm opencv-python matplotlib scikit-learn requests pandas

Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.4.0+cu124 requires torch==2.4.0+cu124, but you have torch 2.4.1 which is incompatible.
You should consider upgrading via the 'd:\Akash\Work\AI\model_evaluation_VSCODE\evaluation\Scripts\python.exe -m pip install --upgrade pip' command.


## Define Functions

In [3]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
import torch    
import cv2
import numpy as np
import requests
import torchvision.transforms as transforms
from pytorch_grad_cam import EigenCAM

from pytorch_grad_cam.utils.image import show_cam_on_image, scale_cam_image
from PIL import Image

COLORS = np.random.uniform(0, 255, size=(80, 3))

def parse_detections(results):
    detections = results.pandas().xyxy[0]
    detections = detections.to_dict()
    boxes, colors, names = [], [], []

    for i in range(len(detections["xmin"])):
        confidence = detections["confidence"][i]
        if confidence < 0.25:
            continue
        xmin = int(detections["xmin"][i])
        ymin = int(detections["ymin"][i])
        xmax = int(detections["xmax"][i])
        ymax = int(detections["ymax"][i])
        name = detections["name"][i]
        category = int(detections["class"][i])
        color = COLORS[category]

        boxes.append((xmin, ymin, xmax, ymax))
        colors.append(color)
        names.append(name)
    return boxes, colors, names


def draw_detections(boxes, colors, names, img):
    for box, color, name in zip(boxes, colors, names):
        xmin, ymin, xmax, ymax = box
        cv2.rectangle(
            img,
            (xmin, ymin),
            (xmax, ymax),
            color, 
            2)

        cv2.putText(img, name, (xmin, ymin - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2,
                    lineType=cv2.LINE_AA)
    return img

def renormalize_cam_in_bounding_boxes(boxes, colors, names, image_float_np, grayscale_cam):
    """Normalize the CAM to be in the range [0, 1] 
    inside every bounding boxes, and zero outside of the bounding boxes. """
    renormalized_cam = np.zeros(grayscale_cam.shape, dtype=np.float32)
    for x1, y1, x2, y2 in boxes:
        renormalized_cam[y1:y2, x1:x2] = scale_cam_image(grayscale_cam[y1:y2, x1:x2].copy())    
    renormalized_cam = scale_cam_image(renormalized_cam)
    eigencam_image_renormalized = show_cam_on_image(image_float_np, renormalized_cam, use_rgb=True)
    image_with_bounding_boxes = draw_detections(boxes, colors, names, eigencam_image_renormalized)
    return image_with_bounding_boxes


## Inference

### Setup

In [4]:
import pathlib
temp = pathlib.PosixPath
pathlib.PosixPath = pathlib.WindowsPath

In [7]:
pathlib.PosixPath = temp

In [7]:
import shutil
from datetime import datetime

torch.hub.set_dir(os.path.join(os.getcwd(), 'models'))
# model = torch.hub.load(os.path.join('models', 'ultralytics', 'yolov5'), 'custom', path = weights_path)
model = torch.hub.load('ultralytics/yolov5', 'custom', path=weights_path)
model.eval()
model.cuda()

model_target_layers = [model.model.model.model[f] for f in target_layers]

images = [os.path.join(input_dir, f) for f in os.listdir(input_dir)]

if not os.path.exists(save_path):
    os.mkdir(save_path)

save_dir = save_dir_name + '_' + datetime.now().strftime("%Y%m%d_%H-%M")

if os.path.exists(os.path.join(save_path, save_dir)):
    shutil.rmtree(os.path.join(save_path, save_dir))
os.mkdir(os.path.join(save_path, save_dir))

for i in images:
    img = np.array(Image.open(i))
    img = cv2.resize(img, (640, 640))
    rgb_img = img.copy()
    img = np.float32(img) / 255
    transform = transforms.ToTensor()
    tensor = transform(img).unsqueeze(0)
    
    results = model([rgb_img])
    boxes, colors, names = parse_detections(results)
    detections = draw_detections(boxes, colors, names, rgb_img.copy())
    cam = EigenCAM(model, model_target_layers)
    grayscale_cam = cam(tensor)[0, :, :]
    cam_image = show_cam_on_image(img, grayscale_cam, use_rgb=True)
    
    renormalized_cam_image = renormalize_cam_in_bounding_boxes(boxes, colors, names, img, grayscale_cam)
    Image.fromarray(np.hstack((rgb_img, cam_image, renormalized_cam_image))).save(os.path.join(save_path, save_dir, img_prefix + os.path.splitext(os.path.basename(i))[0] + img_suffix + os.path.splitext(os.path.basename(i))[1]))
    

Using cache found in d:\Akash\Work\AI\model_evaluation_VSCODE\kaiseki\scripts\XAI\models\ultralytics_yolov5_master
YOLOv5  2024-10-29 Python-3.8.10 torch-2.4.1+cu124 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


NotImplementedError: Could not run 'torchvision::nms' with arguments from the 'CUDA' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'torchvision::nms' is only available for these backends: [CPU, Meta, QuantizedCPU, BackendSelect, Python, FuncTorchDynamicLayerBackMode, Functionalize, Named, Conjugate, Negative, ZeroTensor, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradXLA, AutogradMPS, AutogradXPU, AutogradHPU, AutogradLazy, AutogradMeta, Tracer, AutocastCPU, AutocastXPU, AutocastCUDA, FuncTorchBatched, BatchedNestedTensor, FuncTorchVmapMode, Batched, VmapMode, FuncTorchGradWrapper, PythonTLSSnapshot, FuncTorchDynamicLayerFrontMode, PreDispatch, PythonDispatcher].

CPU: registered at C:\actions-runner\_work\vision\vision\pytorch\vision\t

In [6]:
import torch
print(torch.cuda.is_available())  # Should return True if CUDA is installed and working
print(torch.version.cuda)  # Check the CUDA version

True
12.4


In [1]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


### FFMPEG Video Creation

In [ ]:
import os
import subprocess

img_file_extension = '.jpg'

ffmpeg_cmd = 'ffmpeg -framerate 30 -i ' + os.path.join(save_path, save_dir, img_prefix) + '%06d' + img_suffix + img_file_extension + ' -vcodec libx264 -r 30 ' + os.path.join(save_path, save_dir, save_dir) + '.mp4'

subprocess.call(ffmpeg_cmd, shell=True)